# Word Normalization

### José Pablo Kiesling Lange

In [1]:
import re
from collections import Counter
from tokenizers import Tokenizer, models, trainers, pre_tokenizers

In [2]:
corpus_path = "data/escher_comments.txt"

In [3]:
with open(corpus_path, "r", encoding="utf-8") as file:
    corpus = file.readlines()

In [4]:
corpus = [line.strip() for line in corpus if line.strip()]

## Estandarización

Para ver la efectivdad de la estandarización, se mostrará las 10 palabras más frecuentes del corpus antes y después de la estandarización. El objetivo es ver si hay modificación en la cantidad de dichas palabras o si una nueva palabra aparece con frecuencia.

In [5]:
def get_most_common_words(corpus, n=10):
    words = [word for line in corpus for word in line.split()]
    most_common = Counter(words).most_common(n)
    return most_common

In [6]:
most_common_words = get_most_common_words(corpus, n=10)
most_common_words

[('que', 45),
 ('de', 39),
 ('la', 32),
 ('un', 27),
 ('una', 26),
 ('el', 17),
 ('en', 16),
 ('esfera', 15),
 ('reflejo', 15),
 ('y', 14)]

#### Case Folding

Se hace una función que dado un corpus revisa si posee la misma palabra en mayúscula y minúscula.

In [7]:
def get_upper_lower_words(corpus):
    words = set(word for line in corpus for word in line.split())
    upper_lower_words = set()
    for word in words:
        if word.lower() in words and word.upper() in words:
            upper_lower_words.add(word)
    return upper_lower_words

In [8]:
upper_lower_words = get_upper_lower_words(corpus)
upper_lower_words

{'A', 'a'}

Dado que sí hay por lo menos una palabra que aparece en ambas formas, se procede a hacer un case folding (pasar todas las palabras a minúscula).

In [9]:
def case_folding(corpus):
    return [line.lower() for line in corpus]

#### Remove Punctuation

Dado que en las instrucciones para generar el corpus se pidió mínimo 2 oraciones, se asume que mínimo una de las oraciones tiene puntuación. Por lo tanto, se procede a eliminar la puntuación del corpus.

Esto se hace para evitar que palabras como `la,` o `la.` se cuenten como palabras diferentes a `la`.

In [10]:
def remove_punctuation(corpus):
    return [re.sub(r'[^\w\s]', '', line) for line in corpus]

In [11]:
def standarizate(corpus):
    corpus = case_folding(corpus)
    corpus = remove_punctuation(corpus)
    return corpus

In [12]:
corpus = standarizate(corpus)

### Corpus ya estandarizado

In [13]:
most_common_words = get_most_common_words(corpus, n=10)
most_common_words

[('que', 45),
 ('de', 39),
 ('la', 34),
 ('un', 30),
 ('una', 28),
 ('esfera', 18),
 ('en', 18),
 ('el', 17),
 ('reflejo', 16),
 ('se', 15)]

Como se puede apreciar:
- la palabra `la` se incrementó en `2`ocurrencias
- la palabra `un` se incrementó en `3` ocurrencias
- la palabra `una` se incrementó en `3`ocurrencias
- la palabra `esfera` se incrementó en `3` ocurrencias y subió al `6to` lugar
- la palabra `en` se incrementó en `2` ocurrencias y subió al `7mo` lugar
- la palabra `reflejo` se incrementó en `1` ocurrencia
- la palabra `se` **ingresó** al top 10

Por lo tanto, la estandarización ha tenido un efecto positivo en la, ya que ha incrementado la cantidad de ocurrencias de palabras comunes y ha eliminado las variaciones causadas por puntuación.

Algo a recalcar es la ausencia de la estandarización de palabras con o sin tilde. Esto es porque se entrenará el modelo `BPE` en español.

## BPE

La implementación se hará en base a la implementación de `LangformersBlog`

https://blog.langformers.com/bpe-tokenizer-explained/

En la instanciación del `BpeTrainer` se especifica el tamaño del vocabulario y los tokens especiales. Para definir el tamaño del vocabulario, se debe considerar el tamaño del corpus y la cantidad de palabras que se espera que aparezcan en el corpus. Por lo tanto, se hacen métodos para calcular el tamaño del vocabulario y la cantidad de palabras que se espera que aparezcan en el corpus.

In [14]:
def get_vocab_size(corpus):
    return len(set(word for line in corpus for word in line.split()))

In [15]:
vocab_size = get_vocab_size(corpus)
print(f"Vocabulario estimado por tokens únicos: {vocab_size}")

Vocabulario estimado por tokens únicos: 229


Dado que el corpus es pequeño, se define un tamaño de vocabulario de `300` y los tokens especiales son `[PAD]` y `[UNK]`.

In [16]:
tokenizer = Tokenizer(models.BPE())

In [17]:
trainer = trainers.BpeTrainer(
    vocab_size=300,
    special_tokens=["[PAD]", "[UNK]"]
)

El token `[PAD]` se utiliza para rellenar secuencias de longitud variable, mientras que el token `[UNK]` se utiliza para representar palabras desconocidas o fuera del vocabulario.

Ejemplo:
Si el corpus contiene las palabras `hola`, `mundo`, y `adiós`, el vocabulario podría ser:

```
{
    "[PAD]": 0,
    "[UNK]": 1,
    "hola": 2,
    "mundo": 3,
    "adiós": 4
}
```

Entonces si viniera el texto `hello mundo adiós` y otro texto `hola mundo`, el tokenizador lo convertiría a
1. [1, 3, 4] (donde `1` es el token `[UNK]` para `hello`)
2. [2, 3, 0] (donde `0` es el token `[PAD]` para la secuencia de longitud variable)

Es por ello que el uso de ambos tokens es importante para manejar secuencias de longitud variable y palabras desconocidas. En el caso de `PAD` es útil para rellenar secuencias más cortas, mientras que `UNK` es esencial para manejar palabras que no están en el vocabulario.


In [18]:
tokenizer.train_from_iterator(corpus, trainer)

In [19]:
print("Vocabulary:", tokenizer.get_vocab())
tokenizer.save("tiny_tokenizer.json")

Vocabulary: {'todo ': 205, 'ci': 59, 'em': 216, 'c': 5, 'j': 12, 'ás ': 116, 's ': 90, 'los ': 290, 'iz': 219, 'si': 85, 'cur': 282, 'sí m': 151, 'ser ': 253, 'ver ': 204, 'dentr': 286, 'b': 4, 'él ': 233, 'al': 77, 'pción ': 294, 'ser': 91, 'a': 3, 'oca ': 225, 'endo ': 117, 'et': 177, 'habitación ': 152, 're': 40, 'sostiene una esfera ': 256, 'ilu': 288, 'é': 28, 'alrede': 198, 'en la ': 163, 'ent': 50, 'al ': 113, 'iendo ': 126, 'del ': 114, 'una esfera reflect': 199, 'str': 297, 'an': 55, 'veo ': 120, 'present': 166, 'esfera ': 73, 'habita': 135, 'que es ': 191, 'lo que ': 133, 'lu': 156, 'estra ': 266, 'co': 106, 'entorno ': 271, 'ver': 130, 'muebles ': 254, 'su ': 93, 'mano ': 171, 'n ': 38, 'lo ': 84, 'is': 83, 'tom': 299, 'bles ': 212, 'va ': 96, 'di': 154, 'más ': 139, 'da la ': 215, '[PAD]': 0, 'ct': 124, 'ma ': 182, 'to': 141, 'se ve ': 121, 'de ': 43, 'ti': 65, 'en': 36, 'so': 140, 'está ': 149, 'com': 125, 'g': 9, 'jo ': 220, 'entr': 240, 'refleja ': 162, 'i': 11, 'v': 23,